In [1]:
import pandas as pd
import re
import logging
import numpy as np
import torchmetrics
import flash
from flash.text import TextClassificationData, TextClassifier
from tqdm import tqdm
import torch
from transformers import pipeline
tqdm.pandas()
from sentence_splitter import SentenceSplitter
splitter = SentenceSplitter(language='ru')

C:\Users\vkrin\.conda\envs\lightning\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] Не найдена указанная процедура'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
C:\Users\vkrin\.conda\envs\lightning\lib\site-packages\flash\__init__.py:21: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, tp_name):
C:\Users\vkrin\.conda\envs\lightning\lib\site-packages\flash\__init__.py:21: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, tp_name):


In [2]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.model_selection import train_test_split

In [3]:
def clean_text(text):
    # приводим текст к нижнему регистру
    # создаем регулярное выражение для удаления лишних символов
    regular = r'[\*+\#+\№\"\-+\+\=+\?+\&\^\.+\;\,+\>+\(\)\/+\:\\+]'
    # регулярное выражение для замены ссылки на "URL"
    regular_url = r'(http\S+)|(www\S+)|([\w\d]+www\S+)|([\w\d]+http\S+)'
    # удаляем лишние символы
    text = re.sub(regular, '', text)
    # заменяем ссылки на "URL"
    text = re.sub(regular_url, r'URL', text)
    # заменяем числа и цифры на ' NUM '
    # удаляем лишние пробелы
    text = re.sub(r'[\n\t\s+]', ' ', text)
    # возвращаем очищенные данные
    return text

In [4]:
def expert(x):
    if 'АКРА' in x:
        return 'АКРА'
    elif 'НКР' in x:
        return 'НКР'
    elif 'НРА' in x:
        return 'НРА'
    elif 'Эксперт РА' in x:
        return 'РА'
    else:
        return 'НЗ'

In [5]:
dict_ratings_detail = {
    'A': ((6.33+5.92)/2, 6.33),
    'A+': ((6.75+6.33)/2, 6.75),
    'A-': ((5.92+5.54)/2, 5.92),
    'AA': ((7.68+7.20)/2, 7.68),
    'AA+': ((8.22+7.68)/2, 8.22),
    'AA-': ((7.20+6.75)/2, 7.20),
    'AAA': ((10.0+8.22)/2, 10.0),
    'B': ((2.95+2.56)/2, 2.95),
    'B+': ((3.33+2.95)/2, 3.33),
    'B-': ((2.56+2.16)/2, 2.56),
    'BB': ((4.06+3.7)/2, 4.06),
    'BB+': ((4.42+4.06)/2, 4.42),
    'BB-': ((3.70+3.33)/2, 3.70),
    'BBB+': ((5.54+5.16)/2, 5.54),
    'BBB-': ((4.79+4.42)/2, 4.79),
    'BBB': ((5.16+4.79)/2, 5.16),
    'C': (2.16/2, 2.16),
}

dict_ratings = {
    'A': 6.75,
    'AA': 8.22,
    'AAA': 10.0,
    'B': 3.33,
    'BB': 4.42,
    'BBB': 5.54,
    'C': 2.16,
}

ratings = sorted(dict_ratings.items(), key=lambda x: x[1])
ratings_detail = sorted(dict_ratings_detail.items(), key=lambda x: x[1][1])

In [6]:
data = pd.read_excel('CRA_train_1200.xlsx', index_col=0)
data['cnt'] = data.pr_txt.apply(lambda x: len(x.split()))
data['sign'] = data.apply(lambda x: x['Уровень рейтинга'].replace(x['Категория'],''), axis=1)
data['labels'] = data['Уровень рейтинга'].map(lambda x: dict_ratings_detail[x][0])
data['non_detail_labels'] = data['Категория'].map(lambda x: dict_ratings[x])
data['sign'] = data['sign'].apply(lambda x: x if len(x)>1 else '=')
data['Эксперт'] = data['pr_txt'].progress_apply(expert)
data['text'] = data['pr_txt'].progress_apply(lambda x: [clean_text(i) for i in splitter.split(x)])
data['ОСК'] = data['pr_txt'].progress_apply(lambda x: re.search('ОСК', x.replace('(', ' ').replace(')', ' ')) is not None)
data.head()

100%|███████████████████████████████████████████████████████████████████████████| 1200/1200 [00:00<00:00, 88478.09it/s]


,pr_txt,Категория,Уровень рейтинга,cnt,sign,labels,non_detail_labels,Эксперт,text,ОСК
Id,,,,,,,,,,
1,Повышение кредитного рейтинга Акционерного об...,A,A,1060,=,6.125,6.75,АКРА,[Повышение кредитного рейтинга Акционерного об...,True
2,«Эксперт РА» подтвердил кредитный рейтинг комп...,BB,BB,1007,=,3.880,4.42,РА,[«Эксперт РА» подтвердил кредитный рейтинг ком...,False
3,"НКР повысило кредитный рейтинг ООО ""ОТЭКО-Порт...",A,A,1493,=,6.125,6.75,НКР,[НКР повысило кредитный рейтинг ООО ОТЭКОПортс...,True
4,«Эксперт РА» присвоил кредитный рейтинг ПАО «Ф...,AAA,AAA,1576,=,9.110,10.00,РА,[«Эксперт РА» присвоил кредитный рейтинг ПАО «...,False
5,29 марта 2023 г. Ведущий рейтинговый аналитик ...,BBB,BBB,1161,=,4.975,5.54,НРА,[29 марта 2023 г Ведущий рейтинговый аналитик ...,False


In [8]:
# classifier = pipeline("summarization", model='d0rj/rut5-base-summ', device='cuda')
# data['text'] = data['pr_txt'].progress_apply(lambda x: classifier(x, batch_size=4, max_length=96)[0]['summary_text'])
# data.to_csv('sum_texts')
# data = pd.read_csv('sum_texts', index_col=0)

In [32]:
train, test = train_test_split(data, test_size=0.1, shuffle=True, stratify=data['labels'])
train = train[['text', 'labels']].explode('text')
train['cnt'] = train.groupby('text').transform('count')
exclude_data = set(train[train.cnt>1].text.unique())
train = train[(train.text.map(lambda x: len(x.split()))>10)&(train['cnt']<=2)].drop_duplicates(subset=['text'])
test = test[['text', 'labels']].explode('text')
test = test[~test.text.isin(exclude_data)]

In [31]:
model_args = ClassificationArgs()
model_args.num_train_epochs = 4
model_args.train_batch_size = 48
model_args.regression = True
model_args.overwrite_output_dir=True
model_args.learning_rate=5e-5

model = ClassificationModel(
    "bert",
    "ai-forever/sbert_large_nlu_ru",
    tokenizer_type="bert",
    tokenizer_name="ai-forever/sbert_large_nlu_ru",
    num_labels=1,
    args=model_args
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ai-forever/sbert_large_nlu_ru and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
torch.set_float32_matmul_precision('medium')

In [ ]:
model.train_model(train)

  0%|          | 0/30371 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/633 [00:00<?, ?it/s]

C:\Users\vkrin\.conda\envs\lightning\lib\site-packages\torch\optim\lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Running Epoch 1 of 4:   0%|          | 0/633 [00:00<?, ?it/s]

In [16]:
model.eval_model(test)

  0%|          | 0/5841 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/731 [00:00<?, ?it/s]

({'eval_loss': 1.6892789770159793},
 array([6.79296875, 6.61328125, 6.671875  , ..., 6.578125  , 6.56640625,
        6.828125  ]),
 [{'guid': 0, 'text_a': 'Кредитный рейтинг Публичного акционерного общества «РУСАЛ Братский алюминиевый завод» далее — Компания ПАО «РУСАЛ Братск» базируется на определении оценки собственной кредитоспособности ОСК самой Компании и поддерживающего лица — МКПАО «ОК РУСАЛ» далее — поддерживающее лицо ПЛ «РУСАЛ» Холдинг а также на определении степени поддержки и степени зависимости Компании и поддерживающего лица от однородных факторов риска', 'text_b': None, 'label': 6.54},
  {'guid': 1, 'text_a': 'С учетом высокой степени поддержки и высокой степени зависимости рейтинг Компании присвоен на уровне ОСК поддерживающего лица — a', 'text_b': None, 'label': 6.54},
  {'guid': 2, 'text_a': 'ОСК поддерживающего лица обусловлена сильной рыночной позицией как глобального производителя алюминия и сильной оценкой бизнеспрофиля которая в свою очередь определяется низкой с

In [20]:
import gradio as gr


def mapping_ratings(x):
    for label, (_, r) in ratings_detail:
        if x < r:
            return label

def text_analysis(text):
    sents = [clean_text(i) for i in splitter.split(text) if len(i.split()) > 7]
    outputs, _ = model.predict(sents)
    rating = outputs.mean()
    answer = 'AAA'
    detail_answer = 'AAA'
    pos_tokens = [(i, mapping_ratings(k)) for i, k in zip(sents, outputs)]

    for label, r in ratings:
        if rating < r:
            answer = label
            break
    detail_answer = mapping_ratings(rating)
    
    result = {
        'Уровень кредитного рейтинга': answer,
        'Детализированный уровень кредитного рейтинга': detail_answer,
        'Среднее значение полученных баллов': rating,
    }

    return result, [(i, k) for i,k in pos_tokens if k.startswith(answer)]

In [21]:
demo = gr.Interface(
    text_analysis,
    gr.Textbox(placeholder="Enter text here..."),
    ["json", 
     gr.HighlightedText(show_legend=True, container=True)],
    examples=[
        ["«Эксперт РА» подтвердил рейтинг компании «Мостотрест»..."],
        ["В то же время, устойчивая база государственных..."],
    ],
)

demo.launch(inbrowser=True, height=1200)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [20]:
datamodule = TextClassificationData.from_data_frame(
    'text',
    'Категория',
    train_data_frame=data.explode('text'),
    val_split=0.1,
    batch_size=512,
)

Map:   0%|          | 0/70388 [00:00<?, ? examples/s]

In [21]:
datamodule.labels

['A', 'AA', 'AAA', 'B', 'BB', 'BBB', 'C']

In [12]:
model = TextClassifier(
    num_classes=len(datamodule.labels),
    backbone='intfloat/multilingual-e5-large',
    labels=datamodule.labels,
    metrics=[
        torchmetrics.F1Score(num_classes=len(datamodule.labels), average='weighted'),
        torchmetrics.Accuracy(num_classes=len(datamodule.labels), average='weighted'),
        torchmetrics.Precision(num_classes=len(datamodule.labels), average='weighted'),
        torchmetrics.Recall(num_classes=len(datamodule.labels), average='weighted'),
    ]
)

Using 'intfloat/multilingual-e5-large' provided by Hugging Face/transformers (https://github.com/huggingface/transformers).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at intfloat/multilingual-e5-large and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
torch.set_float32_matmul_precision('medium')

In [14]:
trainer = flash.Trainer(max_epochs=1, gpus=torch.cuda.device_count())
trainer.finetune(model, datamodule=datamodule, strategy='freeze')

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type               | Params
-----------------------------------------------------
0 | train_metrics | ModuleDict         | 0     
1 | val_metrics   | ModuleDict         | 0     
2 | test_metrics  | ModuleDict         | 0     
3 | adapter       | HuggingFaceAdapter | 559 M 
-----------------------------------------------------
1.1 M     Trainable params
558 M     Non-trainable params
559 M     Total params
2,239.574 Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.
